## U-Net model
In this week's assignment, you'll be using a network architecture called "U-Net". The name of this network architecture comes from it's U-like shape when shown in a diagram like this (image from [U-net entry on wikipedia](https://en.wikipedia.org/wiki/U-Net)): 

<img src="U-net_example_wikipedia.png" alt="U-net Image" width="600"/>

U-nets are commonly used for image segmentation, which will be your task in the upcoming assignment. You won't actually need to implement U-Net in the assignment, but we wanted to give you an opportunity to gain some familiarity with this architecture here before you use it in the assignment. 

As you can see from the diagram, this architecture features a series of down-convolutions connected by max-pooling operations, followed by a series of up-convolutions connected by upsampling and concatenation operations. Each of the down-convolutions is also connected directly to the concatenation operations in the upsampling portion of the network. For more detail on the U-Net architecture, have a look at the original [U-Net paper by Ronneberger et al. 2015](https://arxiv.org/abs/1505.04597). 

In this lab, you'll create a basic U-Net using Keras. 

In [27]:
# Import the elements you'll need to build your U-Net
import keras
from keras import backend as K
from keras.engine import Input, Model
from keras.layers import Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU

from keras.layers import Conv3DTranspose as Deconvolution3D 

from keras.optimizers import Adam
from keras.layers.merge import concatenate


# Set the image shape to have the channels in the first dimension
K.set_image_data_format("channels_first")

### The "depth" of your U-Net
The "depth" of your U-Net is equal to the number of down-convolutions you will use. In the image above, the depth is 4 because there are 4 down-convolutions running down the left side including the very bottom of the U.

For this exercise, you'll use a U-Net depth of 2, meaning you'll have 2 down-convolutions in your network. 

### Input layer and its "depth"

In this lab and in the assignment, you will be doing 3D image segmentation, which is to say that, in addition to "height" and "width", your input layer will also have a "length". We are deliberately using the word "length" instead of "depth" here to describe the third spatial dimension of the input so as not to confuse it with the depth of the network as defined above.

The shape of the input layer is `(num_channels, height, width, length)`, where `num_channels` you can think of like color channels in an image, `height`, `width` and `length` are just the size of the input.

For the assignment, the values will be:
- num_channels: 4
- height: 160
- width: 160
- length: 16

## Contracting (downward) path 
Here you'll start by constructing the downward path in your network (the left side of the U-Net).  The `(height, width, length)` of the input gets smaller as you move down this path, and the number of channels increases.

### Depth 0

By "depth 0" here, we're referring to the depth of the first down-convolution in the U-net.

The number of filters is specified for each depth and for each layer within that depth.

The formula to use for calculating the number of filters is:
$$filters_{i} = 32 \times (2^{i})$$

Where $i$ is the current depth.

So at depth $i=0$:
$$filters_{0} = 32 \times (2^{0}) = 32$$


In [28]:
input_layer = Input(shape=(4, 160, 160, 16))

down_depth_0_layer_0 = Conv3D(filters=32, 
                              kernel_size=(3,3,3),
                              padding='same',
                              strides=(1,1,1)
                              )(input_layer)
down_depth_0_layer_0 = Activation('relu')(down_depth_0_layer_0)

down_depth_0_layer_1 = Conv3D(filters=64, 
                kernel_size=(3,3,3),
                padding='same',
                strides=(1,1,1)
               )(down_depth_0_layer_0)
down_depth_0_layer_1 = Activation('relu')(down_depth_0_layer_1)

down_depth_0_layer_pool = MaxPooling3D(pool_size=(2,2,2))(down_depth_0_layer_1)

down_depth_1_layer_0 = Conv3D(filters=64, 
                kernel_size=(3,3,3),
                padding='same',
                strides=(1,1,1)
               )(down_depth_0_layer_pool)

down_depth_1_layer_0 = Activation('relu')(down_depth_1_layer_0)

down_depth_1_layer_1 = Conv3D(filters=128, 
                kernel_size=(3,3,3),
                padding='same',
                strides=(1,1,1)
               )(down_depth_1_layer_0)

down_depth_1_layer_1 = Activation('relu')(down_depth_1_layer_1)



print(input_layer)
print(down_depth_0_layer_0)
print(down_depth_0_layer_1)
print(down_depth_0_layer_pool)
print(down_depth_1_layer_0)
print(down_depth_1_layer_1)

Tensor("input_6:0", shape=(None, 4, 160, 160, 16), dtype=float32)
Tensor("activation_21/Relu:0", shape=(None, 32, 160, 160, 16), dtype=float32)
Tensor("activation_22/Relu:0", shape=(None, 64, 160, 160, 16), dtype=float32)
Tensor("max_pooling3d_3/transpose_1:0", shape=(None, 64, 80, 80, 8), dtype=float32)
Tensor("activation_23/Relu:0", shape=(None, 64, 80, 80, 8), dtype=float32)
Tensor("activation_24/Relu:0", shape=(None, 128, 80, 80, 8), dtype=float32)


## Expanding  (upward) Path

### Concatenate upsampled depth 0 with downsampled depth 0

Now you'll apply a concatenation operation using the layers that are both at the same depth of 0.
- up_depth_0_layer_0: shape is (?, 128, 160, 160, 16)
- depth_0_layer_1: shape is (?, 64, 160, 160, 16)

- Double check that both of these layers have the same height, width and length.
- If they're the same, then they can be concatenated along axis 1 (the channel axis).
- The (height, width, length) is (160, 160, 16) for both.

Run the next cell to check that the layers you wish to concatenate have the same height, width and length.

In [29]:
# Print the shape of layers to concatenate
print(up_depth_0_layer_0)
print(down_depth_0_layer_1)

Tensor("up_sampling3d_7/concat_2:0", shape=(None, 128, 160, 160, 16), dtype=float32)
Tensor("activation_22/Relu:0", shape=(None, 64, 160, 160, 16), dtype=float32)


In [30]:
# Add an upsampling operation to your network
up_depth_0_layer_0 = UpSampling3D(size=(2,2,2))(down_depth_1_layer_1)

# Add a concatenation along axis 1
up_depth_1_concat = concatenate([up_depth_0_layer_0,
                                 down_depth_0_layer_1],
                                axis=1)

# Add a Conv3D up-convolution with 64 filters to your network
up_depth_1_layer_1 = Conv3D(filters=64, 
                            kernel_size=(3,3,3),
                            padding='same',
                            strides=(1,1,1)
                           )(up_depth_1_concat)
up_depth_1_layer_1 = Activation('relu')(up_depth_1_layer_1)

# Add a Conv3D up-convolution with 64 filters to your network
up_depth_1_layer_2 = Conv3D(filters=64, 
                            kernel_size=(3,3,3),
                            padding='same',
                            strides=(1,1,1)
                           )(up_depth_1_layer_1)
up_depth_1_layer_2 = Activation('relu')(up_depth_1_layer_2)

# Add a final Conv3D with 3 filters to your network.
final_conv = Conv3D(filters=3, #3 categories 
                    kernel_size=(1,1,1),
                    padding='valid',
                    strides=(1,1,1)
                    )(up_depth_1_layer_2)

# Add a sigmoid activation to your final convolution.
final_activation = Activation('sigmoid')(final_conv)


print(up_depth_0_layer_0)
print(up_depth_1_concat)
print(up_depth_1_layer_1)
print(up_depth_1_layer_2)
print(final_conv)
print(final_activation)

Tensor("up_sampling3d_8/concat_2:0", shape=(None, 128, 160, 160, 16), dtype=float32)
Tensor("concatenate_8/concat:0", shape=(None, 192, 160, 160, 16), dtype=float32)
Tensor("activation_25/Relu:0", shape=(None, 64, 160, 160, 16), dtype=float32)
Tensor("activation_26/Relu:0", shape=(None, 64, 160, 160, 16), dtype=float32)
Tensor("conv3d_6/BiasAdd:0", shape=(None, 3, 160, 160, 16), dtype=float32)
Tensor("activation_27/Sigmoid:0", shape=(None, 3, 160, 160, 16), dtype=float32)


In [31]:
# Define and compile your model
model = Model(inputs=input_layer, outputs=final_activation)
model.compile(optimizer=Adam(lr=0.00001),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy']
             )

In [32]:
# Print out a summary of the model you created
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 4, 160, 160, 0                                            
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, 32, 160, 160, 3488        input_6[0][0]                    
__________________________________________________________________________________________________
activation_21 (Activation)      (None, 32, 160, 160, 0           conv3d[0][0]                     
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 64, 160, 160, 55360       activation_21[0][0]              
_______________________________________________________________________________________